In [1]:
import faker
import psycopg2
from datetime import datetime
import random

In [2]:
fake = faker.Faker()

def generate_transaction():
    user = fake.simple_profile()

    return {
        "transactionId": fake.uuid4(),
        "userId": user['username'],
        "timestamp": datetime.utcnow().timestamp(),
        "amount": round(random.uniform(10, 1000), 2),        
        'city': fake.city(),
        "country": fake.country(),
        "merchantName": fake.company(),
        "paymentMethod": random.choice(['credit_card', 'debit_card', 'online_transfer']),        
        'affiliateId': fake.uuid4()
    }

def create_table(conn):
    cursor = conn.cursor()

    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS transactions (
            transaction_id VARCHAR(255) PRIMARY KEY,
            user_id VARCHAR(255),
            timestamp TIMESTAMP,
            amount DECIMAL,
            city VARCHAR(255),
            country VARCHAR(255),
            merchant_name VARCHAR(255),
            payment_method VARCHAR(255),       
            affiliateId VARCHAR(255)
        )
        """)

    cursor.close()
    conn.commit()

In [11]:
if __name__ == "__main__":
    conn = psycopg2.connect(
        host='postgres',
        database='financial_db',
        user='postgres',
        password='postgres',
        port=5432
    )

    create_table(conn)

    transaction = generate_transaction()
    cur = conn.cursor()
    print(transaction)

    cur.execute(
        """
        INSERT INTO transactions(transaction_id, user_id, timestamp, amount, city, country, merchant_name, payment_method, affiliateId)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """, (transaction["transactionId"], transaction["userId"], datetime.fromtimestamp(transaction["timestamp"]).strftime('%Y-%m-%d %H:%M:%S'),
              transaction["amount"], transaction["city"], transaction["country"],
              transaction["merchantName"], transaction["paymentMethod"],transaction["affiliateId"])
    )

    cur.close()
    conn.commit()

{'transactionId': '0c7f6466-4dd4-47ef-9585-f6f3f70247a6', 'userId': 'mary07', 'timestamp': 1746865692.825067, 'amount': 209.89, 'city': 'Lisahaven', 'country': 'Congo', 'merchantName': 'Good PLC', 'paymentMethod': 'online_transfer', 'affiliateId': '07727847-3b3d-475a-a2fc-cf510c3cd9fe'}
